# Dog Breed Identification 

The goal of this notebook is to create a classifier to estimate the breed of a dog from a picture. The data and idea comes from [this](https://www.kaggle.com/c/dog-breed-identification) Kaggle Competition. My approach to this problem is to use a pre-trained convolutional neural network for feature extraction, then replace the top of this network with a small neural network which will be trained to classify the images into one of 120 breeds in the training set. 

In [2]:
import numpy as np
import pandas as pd
import os
import keras
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000


/home/connor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [3]:
#keras specific imports 
from keras import Model, Input
from keras.layers import Dense, Flatten,GlobalMaxPool2D,Dropout
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from keras.applications import VGG16,vgg16
from keras.applications import VGG19,vgg19
from keras.applications import InceptionV3, inception_v3

In [4]:
#load pretrained model weights
VGG16_base = VGG16(weights = 'imagenet', include_top = False, input_shape= (224,224,3))
#VGG19_base = VGG19(weights = 'imagenet', include_top = False, input_shape= (224,224,3))
#inception_v3_base = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (300,300,3))

In [5]:
NUMBER_CLASSES = 120

In [6]:
# This will create a generator that the keras model can train on from directories of images that I created earlier in 
# reorganize_images.py. It also will load and preprocess the validation set. 
def create_train_validation(processing_func,image_dim):
    data_gen = ImageDataGenerator(shear_range= 0.3,
                                      horizontal_flip= True,
                                      width_shift_range=0.3,
                                      height_shift_range = 0.3,
                                      preprocessing_function= (lambda x: processing_func(x)) )                                          

    train_generator = data_gen.flow_from_directory('train',
                                                       batch_size = 24,
                                                   target_size= (image_dim,image_dim),
                                                   class_mode= 'categorical')


#creates validation set
    X_val_path, y_val = [], []
    for folder in os.listdir('validation'):
        for path in os.listdir('validation/'+folder):
            y_val.append(folder)
            X_val_path.append('validation/'+folder+'/'+path)

    def prep_from_path(path):
        img = image.load_img(path, target_size = (image_dim,image_dim))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        x = processing_func(x)
        return x.reshape(image_dim,image_dim,3)


    X_val=[]
    for path in X_val_path:
        X_val.append(prep_from_path(path))

    X_val = np.array(X_val)

    class_map = train_generator.class_indices

    y_val = [class_map[y] for y in y_val]

    y_val = to_categorical(y_val)
    return train_generator, X_val, y_val

In [7]:
# creates a model with some number of units in the two hidden layers I am adding
def create_model(base_layers,layer1_units,layer2_units):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = GlobalMaxPool2D()(X)
    X = Dense(layer1_units, activation = 'relu', name = '1')(X)
    X = Dense(layer2_units, activation = 'relu',name = '2')(X)
    predictions = Dense(NUMBER_CLASSES , activation = 'softmax',name = '3')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model

def create_model_no_pooling(base_layers,layer1_units,layer2_units):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = Flatten()(X)
    X = Dropout(.3)(X)
    X = Dense(layer1_units, activation = 'relu', name = '1')(X)
    X = Dense(layer2_units, activation = 'relu',name = '2')(X)
    predictions = Dense(NUMBER_CLASSES , activation = 'softmax',name = '3')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model


In [8]:
# A function to test top-X accuracy which is a common metric for classification problems with a large number of classes
def get_top_X_accuracy(actual,predictions,X= 5):
    actual = np.argmax(actual,axis = 1).reshape(-1,1)
    top_X = np.argsort(predictions, axis = 1)[:,-X:]
    acc = np.any( top_X == actual, axis  = 1)
    return np.mean(acc)

In [45]:
# to prevent overfitting
early_stopping = EarlyStopping(monitor = 'val_loss',
                               min_delta = .01, 
                               patience = 5,
                               verbose = 1,
                               mode = 'min')


In [46]:
image_dim = 224
inception_v3_base = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (image_dim,image_dim,3))
model1 = create_model(inception_v3_base,500,200)

rmsprop = keras.optimizers.RMSprop(lr = .001, decay = .0001)
model1.compile(optimizer=rmsprop,loss = 'categorical_crossentropy',metrics = ['accuracy'])


train_generator, X_val, y_val = create_train_validation(inception_v3.preprocess_input,image_dim)
model1.fit_generator(generator = train_generator, 
                        epochs=40,
                            steps_per_epoch= 50, #there are 24 images per step so this is 1200 per epoch
                            callbacks = [early_stopping],
                            validation_data = (X_val,y_val),
                            workers=8,
                            use_multiprocessing=True,
                            verbose = 1) 


Found 9254 images belonging to 120 classes.
Epoch 1/40
50/50 [==============================] - 27s 533ms/step - loss: 6.2223 - acc: 0.0167 - val_loss: 4.8857 - val_acc: 0.0289
Epoch 2/40
50/50 [==============================] - 19s 376ms/step - loss: 4.7045 - acc: 0.0458 - val_loss: 4.1180 - val_acc: 0.1198
Epoch 3/40
50/50 [==============================] - 20s 405ms/step - loss: 4.3344 - acc: 0.0823 - val_loss: 3.7663 - val_acc: 0.1767
Epoch 4/40
50/50 [==============================] - 19s 381ms/step - loss: 3.8162 - acc: 0.1525 - val_loss: 2.4377 - val_acc: 0.4050
Epoch 5/40
50/50 [==============================] - 19s 381ms/step - loss: 3.4605 - acc: 0.2250 - val_loss: 2.0432 - val_acc: 0.4690
Epoch 6/40
50/50 [==============================] - 19s 379ms/step - loss: 3.1415 - acc: 0.2492 - val_loss: 1.7257 - val_acc: 0.5382
Epoch 7/40
50/50 [==============================] - 19s 380ms/step - loss: 2.8644 - acc: 0.3150 - val_loss: 1.7397 - val_acc: 0.5413
Epoch 8/40
50/50 [=======

In [47]:
preds1 = model1.predict(X_val)

In [48]:
get_top_X_accuracy(y_val,preds1,1)

0.6683884297520661

In [49]:
get_top_X_accuracy(y_val,preds1,5)

0.9328512396694215

In [50]:
get_top_X_accuracy(y_val,preds1,10)

0.9659090909090909

In [37]:
image_dim = 224
inception_v3_base2 = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (image_dim,image_dim,3))
model2 = create_model_no_pooling(inception_v3_base2,500,200)

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-5)
model2.compile(optimizer=sgd,loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [38]:
early_stopping = EarlyStopping(monitor = 'val_loss',
                               min_delta = .01, 
                               patience = 2,
                               verbose = 1,
                               mode = 'min')

train_generator2, X_val2, y_val2 = create_train_validation(inception_v3.preprocess_input,image_dim)

Found 9254 images belonging to 120 classes.


In [39]:
model2.fit_generator(generator = train_generator2, 
                        epochs=50,
                            steps_per_epoch= 50, #there are 24 images per step so this is 1200 per epoch
                            #callbacks = [early_stopping],
                            validation_data = (X_val2,y_val2),
                            workers=8,
                            use_multiprocessing=True,
                            verbose = 1) 

Epoch 1/50
50/50 [==============================] - 26s 525ms/step - loss: 4.8113 - acc: 0.0292 - val_loss: 4.4136 - val_acc: 0.1136
Epoch 2/50
50/50 [==============================] - 19s 384ms/step - loss: 4.5467 - acc: 0.0692 - val_loss: 3.7033 - val_acc: 0.2221
Epoch 3/50
50/50 [==============================] - 19s 385ms/step - loss: 4.1975 - acc: 0.1208 - val_loss: 2.7487 - val_acc: 0.3874
Epoch 4/50
50/50 [==============================] - 19s 384ms/step - loss: 3.7610 - acc: 0.1909 - val_loss: 2.2390 - val_acc: 0.4876
Epoch 5/50
50/50 [==============================] - 19s 385ms/step - loss: 3.3945 - acc: 0.2508 - val_loss: 1.9509 - val_acc: 0.5475
Epoch 6/50
50/50 [==============================] - 19s 385ms/step - loss: 3.0787 - acc: 0.2975 - val_loss: 1.4520 - val_acc: 0.6250
Epoch 7/50
50/50 [==============================] - 19s 386ms/step - loss: 2.9791 - acc: 0.2975 - val_loss: 1.4281 - val_acc: 0.6271
Epoch 8/50
50/50 [==============================] - 20s 402ms/step - 

In [40]:
model2.optimizer.get_config()

{'decay': 9.999999747378752e-06,
 'lr': 0.009999999776482582,
 'momentum': 0.0,
 'nesterov': False}

In [41]:
preds2 = model2.predict(X_val2)

In [42]:
get_top_X_accuracy(y_val2,preds2,1)

0.7458677685950413

In [43]:
get_top_X_accuracy(y_val2,preds2,5)

0.9617768595041323

In [44]:
get_top_X_accuracy(y_val2,preds2,10)

0.9783057851239669

In [24]:
image_dim = 224
vgg16_base = VGG16(weights = 'imagenet', include_top = False, input_shape= (image_dim,image_dim,3))
model3 = create_model(vgg16_base,500,200)
sgd2 = keras.optimizers.SGD(lr = .001,decay = .00001)
early_stopping = EarlyStopping(monitor = 'val_loss',
                               min_delta = .01, 
                               patience = 4,
                               verbose = 1,
                               mode = 'min')

model3.compile(optimizer=sgd2,loss = 'categorical_crossentropy',metrics = ['accuracy'])


train_generator3, X_val3, y_val3 = create_train_validation(vgg16.preprocess_input,image_dim)
model3.fit_generator(generator = train_generator2, 
                        epochs=40,
                            steps_per_epoch= 50, #there are 24 images per step so this is 1200 per epoch
                            callbacks = [early_stopping],
                            validation_data = (X_val3,y_val3),
                            workers=12,
                            use_multiprocessing=True,
                            verbose = 1) 


Found 9254 images belonging to 120 classes.
Epoch 1/40
50/50 [==============================] - 892s 18s/step - loss: 15.7293 - acc: 0.0175 - val_loss: 15.7133 - val_acc: 0.0217
Epoch 2/40
50/50 [==============================] - 875s 18s/step - loss: 15.7278 - acc: 0.0200 - val_loss: 15.7620 - val_acc: 0.0176
Epoch 3/40
50/50 [==============================] - 876s 18s/step - loss: 15.7385 - acc: 0.0208 - val_loss: 15.5488 - val_acc: 0.0320
Epoch 4/40
50/50 [==============================] - 873s 17s/step - loss: 15.5395 - acc: 0.0333 - val_loss: 15.4765 - val_acc: 0.0362
Epoch 5/40
50/50 [==============================] - 883s 18s/step - loss: 15.4080 - acc: 0.0367 - val_loss: 15.3989 - val_acc: 0.0413
Epoch 6/40
50/50 [==============================] - 878s 18s/step - loss: 15.2977 - acc: 0.0425 - val_loss: 15.4100 - val_acc: 0.0372
Epoch 7/40
50/50 [==============================] - 861s 17s/step - loss: 15.2961 - acc: 0.0417 - val_loss: 15.3198 - val_acc: 0.0424
Epoch 8/40
50/50 [

Process ForkPoolWorker-209:
Process ForkPoolWorker-212:
Process ForkPoolWorker-208:
Process ForkPoolWorker-206:
Traceback (most recent call last):
Process ForkPoolWorker-207:
Process ForkPoolWorker-205:
Process ForkPoolWorker-210:
Process ForkPoolWorker-204:
Process ForkPoolWorker-203:
Process ForkPoolWorker-201:
Process ForkPoolWorker-202:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-211:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/conn

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

Process ForkPoolWorker-223:
Process ForkPoolWorker-222:
Process ForkPoolWorker-219:
Process ForkPoolWorker-224:
Process ForkPoolWorker-221:
Process ForkPoolWorker-218:
Process ForkPoolWorker-217:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-220:
Traceback (most recent call last):
Process ForkPoolWorker-214:
Process ForkPoolWorker-213:
Process ForkPoolWorker-216:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-215:
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/connor/anaconda3/lib/python3.6/

KeyboardInterrupt
KeyboardInterrupt


In [64]:
for layer in model1.layers:
    print(layer, layer.trainable, layer.output_shape)

<keras.engine.topology.InputLayer object at 0x7fe0af49a0f0> False (None, 224, 224, 3)
<keras.layers.convolutional.Conv2D object at 0x7fe0aedb3c50> False (None, 111, 111, 32)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aec99588> False (None, 111, 111, 32)
<keras.layers.core.Activation object at 0x7fe0aedb3d30> False (None, 111, 111, 32)
<keras.layers.convolutional.Conv2D object at 0x7fe0aeb97048> False (None, 109, 109, 32)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aec0a780> False (None, 109, 109, 32)
<keras.layers.core.Activation object at 0x7fe0aeba9320> False (None, 109, 109, 32)
<keras.layers.convolutional.Conv2D object at 0x7fe0aeb5b048> False (None, 109, 109, 64)
<keras.layers.normalization.BatchNormalization object at 0x7fe0aeb43be0> False (None, 109, 109, 64)
<keras.layers.core.Activation object at 0x7fe0aeae3710> False (None, 109, 109, 64)
<keras.layers.pooling.MaxPooling2D object at 0x7fe0aea80be0> False (None, 54, 54, 64)
<keras.layers.c